In [2]:
import pandas as pd
import numpy as np
import gensim
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D, Dropout, Conv1D, Input

In [3]:
df = pd.read_csv('combined-selftext.csv')
#df.head()

In [4]:
def str_join(df, sep, *cols):
   ...:     from functools import reduce
   ...:     return reduce(lambda x, y: x.astype(str).str.cat(y.astype(str), sep=sep), 
   ...:                   [df[col] for col in cols])
   ...: 

In [5]:
df['text'] = str_join(df," ", 'title', 'usertext')

In [6]:
df['text'] = str_join(df," ", 'title', 'usertext')
del df['title']
del df['usertext']
df.rename(columns = {'y':'is_suicide'}, inplace = True)

In [7]:
df['text_clean'] = df['text'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['text_clean'], df['is_suicide'], test_size=0.2)

In [9]:
w2v_model = gensim.models.Word2Vec(df['text_clean'],
                                   vector_size=300,
                                   epochs=20,
                                   window=10,
                                   min_count=70)

In [10]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

<ipython-input-10-a03581770919>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-10-a03581770919>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


In [11]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(300, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(300, dtype=float))

In [13]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [14]:
X_train = np.array(X_train_vect_avg)
X_test = np.array(X_test_vect_avg)

In [15]:
print(X_train.shape, X_test.shape)

(1498, 300) (375, 300)


In [16]:
print(y_train.shape, y_test.shape)

(1498,) (375,)


In [17]:
X_train.shape[0]

1498

In [18]:
X_train.shape[1]

300

In [25]:
model4 = keras.Sequential()

model4.add(keras.layers.Input(shape=(300, 1)))
model4.add(keras.layers.Conv1D(5, (2,), padding='same', activation='relu'))
model4.add(keras.layers.Dropout(0.5))
model4.add(keras.layers.Flatten())
model4.add(keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'))
model4.add(keras.layers.Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy',
                              tf.keras.metrics.Precision(),
                              tf.keras.metrics.Recall(),
                              tfa.metrics.F1Score(num_classes=1)])
model4.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 300, 5)            15        
                                                                 
 dropout_2 (Dropout)         (None, 300, 5)            0         
                                                                 
 flatten_2 (Flatten)         (None, 1500)              0         
                                                                 
 dense_4 (Dense)             (None, 64)                96064     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 96,144
Trainable params: 96,144
Non-trainable params: 0
_________________________________________________________________


In [26]:
history4 = model4.fit(
    X_train, y_train, 
    epochs=10, 
    batch_size=32, 
    validation_split=0.2, 
    verbose=1, 
    shuffle=True
)
model4.evaluate(X_test, y_test)

Epoch 1/10
38/38 [==============================] - 1s 13ms/step - loss: 0.6515 - accuracy: 0.6302 - precision_2: 0.6409 - recall_2: 0.6440 - f1_score: 0.6806 - val_loss: 0.6380 - val_accuracy: 0.6200 - val_precision_2: 0.6552 - val_recall_2: 0.5975 - val_f1_score: 0.6928
Epoch 2/10
38/38 [==============================] - 0s 4ms/step - loss: 0.6106 - accuracy: 0.6644 - precision_2: 0.6736 - recall_2: 0.6780 - f1_score: 0.6806 - val_loss: 0.6385 - val_accuracy: 0.6400 - val_precision_2: 0.6321 - val_recall_2: 0.7673 - val_f1_score: 0.6928
Epoch 3/10
38/38 [==============================] - 0s 4ms/step - loss: 0.6072 - accuracy: 0.6636 - precision_2: 0.6646 - recall_2: 0.7023 - f1_score: 0.6806 - val_loss: 0.6353 - val_accuracy: 0.6433 - val_precision_2: 0.6444 - val_recall_2: 0.7296 - val_f1_score: 0.6928
Epoch 4/10
38/38 [==============================] - 0s 4ms/step - loss: 0.6056 - accuracy: 0.6611 - precision_2: 0.6667 - recall_2: 0.6861 - f1_score: 0.6806 - val_loss: 0.6371 - val_

[0.5697172284126282,
 0.7120000123977661,
 0.742514967918396,
 0.6560846567153931,
 array([0.6702128], dtype=float32)]